# Import Functions and Libray

In [ ]:
import BAC0
import time
import os, sys
import paho.mqtt.client as mqtt
import uuid
import sqlite3
import datetime

# Create Connection with SQLite Database and Create Table to Log BACnet Device Data

In [ ]:
conn = sqlite3.connect('C:\\Users\\PARROT\\Desktop\\BACnetData.db')
c = conn.cursor()
c.execute("CREATE TABLE IF NOT EXISTS BACnet_LOG (id integer PRIMARY KEY, TimeStamp TEXT, TAG TEXT, Value TEXT)")
conn.commit()

In [ ]:
# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

# Function to Return BACnet Device Details for Connection

In [ ]:
def get_bac0_device(device, network):
    print("Get BAC0 device")
    name, vendor, address, device_id = device
    return BAC0.device(address, device_id, network, poll=0,object_list=None)

# Main function to connect with BACnet Device, 

In [ ]:
def main():

    # BAC0 Setup
    bacnet = BAC0.connect()
    bacnet.discover()
    # Use first device from list of discoverd devices
    device = bacnet.devices[0]
    # Fetch All info of BACnet Device like Device Name, Device Vendor Details , Device Address, Device ID 
    bac0_device = get_bac0_device(device, bacnet)
    
    # Main Loop of Reading and Logging TAG data
    while True:

        # Get list of points(Tags) with their latest values
        points = bac0_device.points

        # Ignor this code.
        blockPrint()
        print(points)
        enablePrint()

        for point in points:
            timestamp = datetime.datetime.now()
            # point.properties.name > Will get you the BACnet Device Tag Value
            # point.lastValue > Will get you the BACnet Device Tag Value
            # Log everything into SQLite Database
            c.execute("INSERT INTO BACnet_LOG (TimeStamp, TAG, Value) VALUES (?, ?, ?)", (timestamp, point.properties.name, point.lastValue))
            # Commit.
            conn.commit()
            
        # Wait for 4 seconds before sending updated data
        time.sleep(4)

In [ ]:
if __name__ == "__main__":
    # Run Main Function
    main()